In [1]:
import tensorflow as tf
import argparse
import time
import pickle
import numpy as np
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"

from utils import DataLoader
from BiRNNLayer import BiRNNLayer
from PointerLayer import PointerLayer
from AttentionLayer import AttentionLayer

data_dir = 'data'
output_keep_prob = 1.0
input_keep_prob = 1.0
batch_size = 10
num_epochs = 10
rnn_size = 128
learning_rate = 0.002
decay_rate = 0.97

data_loader = DataLoader(data_dir, batch_size)
data_loader.create_batches()
max_para_length = data_loader.max_para_length
max_ques_length = data_loader.max_ques_length
vocab_size = data_loader.vocab_size

#Initialize layers
paragraph_layer = BiRNNLayer(vocab_size, batch_size=batch_size, scope="paraBiRNN")
question_layer = BiRNNLayer(vocab_size, batch_size=batch_size, scope="quesBiRNN")
attention_layer = AttentionLayer(paragraph_layer.outputs, question_layer.outputs, batch_size=batch_size, hidden_size=rnn_size, scope="attention")
output_layer = PointerLayer(attention_layer.outputs, batch_size=batch_size, scope="pointer")

loading preprocessed data...
loading vocab file...
building BiRNNLayer paraBiRNN
building BiRNNLayer quesBiRNN
building attention layer attention
building pointer layer pointer


In [2]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for e in range(num_epochs):
        sess.run(tf.assign(output_layer.learning_rate, learning_rate * (decay_rate ** e)))
        for b in range(data_loader.num_batches):
            start = time.time()
            paragraphs, p_length, questions, q_length, targets_start, targets_end = data_loader.next_batch_variable_seq_length()
            print([len(p) for p in paragraphs])
            feed = {paragraph_layer.inputs: paragraphs,
                    paragraph_layer.seq_length: p_length,
                    question_layer.inputs: questions,
                    question_layer.seq_length: q_length,
                    output_layer.targets_start: targets_start, 
                    output_layer.targets_end: targets_end}
            train_loss, p1, p2, a, b, c, _ = sess.run([output_layer.cost, output_layer.p1, output_layer.p2, output_layer.a, output_layer.b, output_layer.c, output_layer.train_op], feed)

            end = time.time()
            print("{}/{} (epoch {}), train_loss = {:.3f}, time/batch = {:.3f}"
                  .format(e * data_loader.num_batches + b,
                          num_epochs * data_loader.num_batches,
                          e, train_loss, end - start))

heyyy???
572fa64db2c2fd14005682ab
573033c3a23a5019007fcf72
572e7aa8dfa6aa1500f8d00f
5727b1adff5b5019007d92c1
5726d11ef1498d1400e8ec2e
5727b2b2ff5b5019007d92d2
56df10063277331400b4d92b
5726e5c9f1498d1400e8ef3a
5706477d75f01819005e7af8
572f618b04bcaa1900d768a6
248 17
[248, 248, 248, 248, 248, 248, 248, 248, 248, 248]
[ 0.          0.0039733   0.0040125   0.          0.          0.00408616
  0.          0.00408359  0.          0.        ]/87600 (epoch 0), train_loss = inf, time/batch = 0.902
heyyy???
572ba75ff75d5e190021fe72
56d6477c1c85041400947069
57312bcd05b4da19006bce2f
5725e762271a42140099d2ff
572cbf35dfb02c14005c6c20
56fb673d8ddada1400cd63d1
56d3686b59d6e41400146307
56d5d92a1c85041400946e2f
5727ba3c2ca10214002d94cc
572f618b04bcaa1900d768a6
239 18
[239, 239, 239, 239, 239, 239, 239, 239, 239, 239]


InvalidArgumentError: seq_lens(1) < 0
	 [[Node: quesBiRNN_1/cell_0/bidirectional_rnn/bw/ReverseSequence = ReverseSequence[T=DT_FLOAT, Tlen=DT_INT32, batch_dim=0, seq_dim=1, _device="/job:localhost/replica:0/task:0/cpu:0"](embedding_lookup_1, Cast_1)]]

Caused by op 'quesBiRNN_1/cell_0/bidirectional_rnn/bw/ReverseSequence', defined at:
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "c:\users\administrator\appdata\local\programs\python\python35\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "c:\users\administrator\appdata\local\programs\